# Group Project - Advanced Model

Divam Arora, Connor Moore, Hemanth Velan

DSBA 6165

### Sources:
* https://huggingface.co/datasets/gigaword
* https://huggingface.co/docs/datasets/process#export
* https://huggingface.co/docs/datasets/v1.11.0/splits.html
* https://www.geeksforgeeks.org/string-punctuation-in-python/
* https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html
* https://stackoverflow.com/questions/42135409/removing-a-character-from-entire-data-frame
* https://stackoverflow.com/questions/28986489/how-to-replace-text-in-a-string-column-of-a-pandas-dataframe
* https://stackoverflow.com/questions/41425945/python-pandas-error-missing-unterminated-subpattern-at-position-2
* https://aparnamishra144.medium.com/how-to-change-string-data-or-text-data-of-a-column-to-lowercase-in-pandas-248a8ce4ae01

First we need to re-run the code from our EDA/pre-processing notebook that loads and prepares our dataset for implementation.

In [ ]:
# import needed packages
import nltk
import time
import random
import string
import pandas as pd
import datasets as ds
from evaluate import load
from transformers import BartTokenizer, BartForConditionalGeneration

# download stop word package from nltk library
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

### Dataset
Because our dataset is pulled directly from Huggingface's datasets library, there is no need for a local copy of the data. Running the cell below creates an instance of the specified dataset in your workspace environment.

In [ ]:
# https://huggingface.co/datasets/gigaword
# https://huggingface.co/docs/datasets/v1.11.0/splits.html

# download gigaword dataset from Hugging Face dataset library
train, test, validation = ds.load_dataset("gigaword", split=["train", "test", "validation"])

In [ ]:
# display the dataset splits
print(train)
print(test)
print(validation)

In [ ]:
# https://huggingface.co/docs/datasets/process#export

# export the training dataset to a pandas dataframe and display
df_train = train.to_pandas()
print("Train df exported.")

# export the test dataset to a pandas dataframe
df_test = test.to_pandas()
print("Test df exported.")

# export the validation dataset to a pandas dataframe
df_val = validation.to_pandas()
print("Validation df exported.")

### Balancing the train-test split
The standard provided division between train, test, and validation is extremely unbalanced towards train (95%), and the dataset overall is far too large to run through our model in a reasonable timespan. We decided to shrink the train set to 70,000 entries, and concat the provided test and validation sets. From that combined test-val set we will extract a 25,000-entry test set and a 5,000 entry validation set.

In [ ]:
# select 70,000 rows randomly from the train dataframe

df_train_short = df_train.sample(n = 70000, random_state=2, ignore_index=True)

df_train_short

In [ ]:
# combine provided test and val sets and reseparate randomly into smaller subsets

# concat test and validation sets
test_val = [df_test, df_val]
df_testval_bulk = pd.concat(test_val)

# take a random sample of 30000 rows from the test and validation bulk set
df_testval_short = df_testval_bulk.sample(n = 30000, random_state=3, ignore_index=True)

# take a random 5000 row sample from the test-val subset
df_val_short = df_testval_short.sample(n = 5000, random_state=4, ignore_index=True)

# drop all rows taken for the validation sample from the test-val subset to create the test set
df_test_short = df_testval_short.drop(df_val_short.index, axis=0)

### Data Pre-Processing
We decided to truncate our pre-processing pipeline slightly from our baseline model notebook (removing lemmatization and not vectorizing text prior to passing it to the BART pretrained tokenizer) because BART models are designed to accept full, grammatically correct sentences. We thought passing more "normal" text during training may give the model better context and improve learning.

In [ ]:
# the methods required to perform this function were found in this article -
# https://aparnamishra144.medium.com/how-to-change-string-data-or-text-data-of-a-column-to-lowercase-in-pandas-248a8ce4ae01
# the function and comments are our original work

# set all words in all rows to lower case

def lower(df):
    # vectorize strings in each row in summary column and set to lower case
    df["summary"] = df["summary"].str.lower()
    print("summary column lowercased")
    # vectorize strings in each row in document column and set to lower case
    df["document"] = df["document"].str.lower()
    print("document column lowercased")

In [ ]:
# geeks for geeks and pandas doc pages were used as template source code and informed about parameter options
# stackoverflow posts helped with debugging issues
# https://stackoverflow.com/questions/42135409/removing-a-character-from-entire-data-frame
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html
# https://www.geeksforgeeks.org/string-punctuation-in-python/
# https://stackoverflow.com/questions/41425945/python-pandas-error-missing-unterminated-subpattern-at-position-2
# https://stackoverflow.com/questions/28986489/how-to-replace-text-in-a-string-column-of-a-pandas-dataframe
# comments and function are our original work, source code was modifed to fit our workspace

# remove all symbols and punctuation

# create instance of all punctuation symbols
punctuation = string.punctuation

# since we learned there are lots of apostrophe s in the dataset during EDA, we will add this to our remove list
punct_list = ["'s"]

# add all punctuation from the premade variable to our new list
for symbol in punctuation:
    punct_list.append(symbol)

# display the symbols included in our list
print(punct_list)

def remove_punctuation(df):
    # for each symbol in our punctuation list
    for symbol in punct_list:
        # iterate through the dataframe and replace every instance of the symbol with an empty string
        df["document"] = df["document"].str.replace(symbol, "", regex=False)
        df["summary"] = df["summary"].str.replace(symbol, "", regex=False)
    print("symbols removed")

In [ ]:
# create data pre-processing pipeline

def pre_proc(df):
    # lowercase
    lower(df)
    # remove punctuation and symbols
    remove_punctuation(df)
    print("pre-processed successfully")

In [ ]:
# call the data pre-processing pipeline for each of the dataset splits
# minus the lemmatization and tokenizer steps from the baseline notebook

pre_proc(df_train_short)
print("train df completed")
pre_proc(df_val_short)
print("test df completed")
pre_proc(df_test_short)
print("validation df completed")

# display new format of data using training set
df_train_short.head()

Our dataset splits are now pre-processed and ready for use with models.

# New attempt at fine-tuning / advanced model
Using same approach utilized in baseline model notebook, with modifications and adjustments made to runBART() function to eliminate null value model output issues and attempt to improve hyperparameters.

In [ ]:
# tokenizer and model loading for bart-base

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
# new version OF runBart function, updated from our baseline model for improved speed and performance
# testing and trial and error showed that removing the component of the function that calculated a unique maxlength for every summary sped up prediction generation significantly
# in the baseline model we had issues with large numbers of NaN empty summary predictions,
# through some research, testing, and trial and error we were able to find a configuration for the encode and generate() steps that corrected this issue

def runBart(df):

    # Empty lists for predictions and performance timestamps
    predictions = []
    times = []

    # For the number of rows in the given dataframe
    for i in range(len(df)):
        # Create a start timestamp
        start = time.perf_counter()

        # Create a document instance using the row's entry for the stringified document
        doc = df.iloc[i]["document"]

        # Encoding inputs using BART tokenizer 
        inputs = tokenizer.encode(doc, return_tensors='pt', max_length=1024, truncation=True)

        # Generate vectorized summary using encoded inputs
        summary_ids = model.generate_text(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

        # Decode the summary into a human-readable format
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Append the predicted summary to a list of predictions
        predictions.append(summary)

        # Create an end timestamp
        end = time.perf_counter()

        # Calculate computation speed
        speed = end - start

        # Append computation speed to list
        times.append(speed)

        # If the iteration is a multiple of 1000
        if i % 1000 == 0:
            # Calculate the average computation time per row so far and print
            avg_time = sum(times) / len(times)
            print("Average time per row at", i, "row:", avg_time)

    # Create a new column for the dataframe using the predictions generated and return the modified dataframe
    df["BART_Pred"] = predictions
    return df

In [ ]:
# run the model to generate predictions using the test set
runBart(df_test_short)

df_test_short

In [ ]:
# check generated predictions for NaN values

# count null values in BART pred columnm
null_predictions = df_test_short['BART_Pred'].isna().sum()

print("there were", null_predictions, "empty predictions generated.")

# BERTScore Metrics

In [ ]:
# initialize BERTScore metric

bertscore = load("bertscore")

In [ ]:
# generating BERTScore metrics for model predictions

# create list of prediction outputs
test_predictions = list(df_test_short["BART_Pred"].astype(str))
# create list of true outputs
test_references = list(df_test_short["summary"].astype(str))
# calculate BERTScore values comparing model predictions with true summaries
test_results_bert = bertscore.compute(predictions=test_predictions, references=test_references, lang="en")

In [ ]:
# calculate average precision, recall and F1 scores from BERTScore model based on model predictions

# create list of result keys
test_keys = list(test_results_bert.keys())

# for number of values in keylist-1
for k in range(len(test_keys)-1):
    # sum total of all result values
    s_test = sum(test_results_bert[test_keys[k]])
    # calculate the total number of result values
    le_test = len(test_results_bert[test_keys[k]])
    # compute average result value
    avg_test = s_test/le_test

    print("test set result:")
    print("Average {} is {}".format(test_keys[k], avg_test))

# ROUGE Metrics

In [ ]:
# initialize ROUGE metrics model

rouge = load('rouge')

In [ ]:
# generate ROUGE metrics scores for train and test model outputs

# compute ROUGE metrics scores comparing model predictions with true outputs
test_results_rouge = rouge.compute(predictions=test_predictions, references=test_references)

print("test set results:")
print(test_results_rouge)

# Observe a few specific model predictions for human evaluation

In [ ]:
# produce five random input/target/prediction pairs from the model

# empty list for index sample
indexList = []

# generate five random index values and add to list
for i in range(5):
    index = random.randint(0,25000)
    indexList.append(index)

print(indexList)
print()

# for index in list
for ind in indexList:
    # display the original input, the target summary, and what the model predicted for a summary
    print("index:", ind)
    print("original input:", df_test_short.iloc[ind]["document"])
    print("target summary:", df_test_short.iloc[ind]["summary"])
    print("model prediction:", df_test_short.iloc[ind]["BART_Pred"])
    print()